# LIDC IDRI 2D SEGMENTATION WITH TERNARY CLASSES

## Import Libraries

In [1]:
#!pip list

In [2]:
import pandas as pd
import argparse
import os
from collections import OrderedDict
from glob import glob
import yaml
import numpy as np

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms.functional as TF
from torchvision import transforms
import torchsummary as summary

import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

import matplotlib.pyplot as plt

from Unet_new.unet_model import UNet
from UnetNested.Nested_Unet import NestedUNet

## Define Parameters

In [3]:
name = "UNet"           #default = "UNet"; can be NestedUNet
epochs = 100            #default = 400
batch_size = 12         #default = 12
early_stopping = 50     #default = 50
num_workers = 8         #default = 8
optimizer = 'Adam'      #default = 'Adam'; can be SGD
lr = 1e-5               #default = 1e-5
momentum = 0.9          #default = 0.9
weight_decay = 1e-4     #default = 1e-4
nesterov = False        #default = False
augmentation = True     #default = False

## Define Functions

### Dataset

In [4]:
class LidcDataset(Dataset):
    def __init__(self, IMAGES_PATHS, MASK_PATHS, transforms):
        self.image_paths = IMAGES_PATHS
        self.mask_paths = MASK_PATHS
        
        self.transforms = transforms

    def __getitem__(self, index):
        image = np.load(self.image_paths[index])
        mask = np.load(self.mask_paths[index])

        #Make image and mask 3 dimensional
        image = image.reshape(512,512,1)
        mask = mask.reshape(512,512,1)

        #Convert datatype
        mask = mask.astype('uint8')

        #Apply augmentation
        augmented = self.transforms(image=image,mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        
        mask = mask.reshape([1,512,512])

        image, mask = image.type(torch.FloatTensor), mask.type(torch.FloatTensor)     

        return image, mask
    
    def __len__(self):
        return len(self.image_paths)

In [5]:
toTensor = A.Compose([
            ToTensorV2()
        ])

transform1 = A.Compose([
            A.Rotate(limit=[90, 90], interpolation=1, border_mode=4, value=None, mask_value=None, p=1),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Affine(scale=None, translate_percent=None, translate_px=None, rotate=None, shear=[-45, 45], interpolation=1, mask_interpolation=0, cval=0, cval_mask=0, mode=0, fit_output=False, keep_ratio=False, always_apply=False, p=1),
            ToTensorV2()
        ])

transform2 = A.Compose([
            A.Rotate(limit=[180, 180], interpolation=1, border_mode=4, value=None, mask_value=None, p=1),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Affine (scale=None, translate_percent=None, translate_px=None, rotate=None, shear=[-45, 45], interpolation=1, mask_interpolation=0, cval=0, cval_mask=0, mode=0, fit_output=False, keep_ratio=False, always_apply=False, p=1),
            ToTensorV2()
        ])

transform3 = A.Compose([
            A.Rotate (limit=[270, 270], interpolation=1, border_mode=4, value=None, mask_value=None, p=1),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Affine (scale=None, translate_percent=None, translate_px=None, rotate=None, shear=[-45, 45], interpolation=1, mask_interpolation=0, cval=0, cval_mask=0, mode=0, fit_output=False, keep_ratio=False, always_apply=False, p=1),
            ToTensorV2()
        ])

### Metrics

In [6]:
def iou_score_multiclass(output, target, n_classes):
    output = torch.nn.functional.softmax(output, dim=1)
    output = torch.argmax(output, dim=1).squeeze(1)
    target = torch.argmax(target, dim=1)
    iou_list = list()
    curr_iou_list = list()

    output = output.view(-1)
    target = target.view(-1)

    for sem_class in range(n_classes):
        output_inds = (output == sem_class)
        target_inds = (target == sem_class)

        if target_inds.long().sum().item() == 0:
            iou_curr = float('nan')
        else:
            intersection_curr = (output_inds[target_inds]).long().sum().item()
            union_curr = output_inds.long().sum().item() + target_inds.long().sum().item() - intersection_curr
            iou_curr = float(intersection_curr) / float(union_curr)
            curr_iou_list.append(iou_curr)
        iou_list.append(iou_curr)

    return np.mean(curr_iou_list)

# def dice_coef(output, target):
#     smooth = 1e-5
#     target_f = target.flatten()
#     output_f = output.flatten()
#     intersection = np.sum(target_f * output_f)
#     return (2. * intersection + smooth) / (np.sum(target_f) + np.sum(output_f) + smooth)

def dice_coef_multiclass(output, target, n_classes):
    smooth = 1e-5 
    output = torch.nn.functional.softmax(output, dim=1)
    output = torch.argmax(output, dim=1).squeeze(1)
    target = torch.argmax(target, dim=1)
    intersection = (output*target).sum()

    dice = (2. * intersection + smooth) / \
        (output.sum() + target.sum() + smooth)
    return dice.item()

    # dice = 0
    # for i in range(n_classes):
    #     dice += dice_coef(output[:,i,:,:], target[:,:,:])
    # return dice/n_classes

def sensitivity_metric_multiclass(output, target):
    eps = 1e-5
    output = torch.nn.functional.softmax(output, dim=1)
    output = torch.argmax(output, dim=1).squeeze(1)
    target = torch.argmax(target, dim=1)
    # elements of confusion matrix
    tp = torch.sum(output * target) # True Positive
    fp = torch.sum(output * (1 - target)) # False Positive
    fn = torch.sum((1 - output) * target) # False Negative
    tn = torch.sum((1 - output) * (1 - target)) # True Negative
    # compute sensitivity
    sensitivity = (tp + eps) / (tp + fn + eps)
    
    return sensitivity.item()


def dice_coef2(output, target):
    "This metric is for validation"
    smooth = 1e-5
    output = output.view(-1)
    output = (output>0.5).float().cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output*target).sum()

    return (2. * intersection + smooth) / \
        (output.sum() + target.sum() + smooth)

In [7]:
# def iou_score(output, target):
#     smooth = 1e-5
#     if torch.is_tensor(output):
#         output = torch.sigmoid(output).data.cpu().numpy()
#     if torch.is_tensor(target):
#         target = target.data.cpu().numpy()
#     output_ = output > 0.5
#     target_ = target > 0.5
#     intersection = (output_ & target_).sum()
#     union = (output_ | target_).sum()

#     return (intersection + smooth) / (union + smooth)


# def dice_coef(output, target):
#     smooth = 1e-5
#     target_f = target.flatten()
#     output_f = output.flatten()
#     intersection = np.sum(target_f * output_f)
#     return (2. * intersection + smooth) / (np.sum(target_f) + np.sum(output_f) + smooth)
#     #Sigmoid is used because the U-Net output is logit
#     # output = torch.sigmoid(output).view(-1).data.cpu().numpy()
#     # target = target.view(-1).data.cpu().numpy()
#     # intersection = (output*target).sum()

#     # return (2. * intersection + smooth) / \
#     #     (output.sum() + target.sum() + smooth)


# def sensitivity_metric(output, target):
#     eps = 1e-5
#     output = torch.sigmoid(output).view(-1).data.cpu()
#     target = target.view(-1).data.cpu()
#     # elements of confusion matrix
#     tp = torch.sum(output * target) # True Positive
#     fp = torch.sum(output * (1 - target)) # False Positive
#     fn = torch.sum((1 - output) * target) # False Negative
#     tn = torch.sum((1 - output) * (1 - target)) # True Negative
#     # compute sensitivity
#     sensitivity = (tp + eps) / (tp + fn + eps)
    
#     return sensitivity.item()


# def dice_coef2(output, target):
#     "This metric is for validation"
#     smooth = 1e-5
#     output = output.view(-1)
#     output = (output>0.5).float().cpu().numpy()
#     target = target.view(-1).data.cpu().numpy()
#     intersection = (output*target).sum()

#     return (2. * intersection + smooth) / \
#         (output.sum() + target.sum() + smooth)

### Utilities

In [8]:
def str_to_bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class AverageMeter(object):
    #Computes and stores the average and current value
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Get Configuration

In [9]:
if augmentation == True:
    file_name = name + '_with_augmentation'
else:
    file_name = name + '_base'
os.makedirs('model_outputs/{}'.format(file_name), exist_ok=True)
print("Creating directory called ", file_name)

print('-' * 20)
print("Configuration Setting: ")
print("Model: ", name)
print("Max Epochs: ", epochs)
print("Batch Size: ", batch_size)
print("Number of Workers: ", num_workers)
print("Optimizer: ", optimizer)
print("Learning Rate: ", lr)
print("Augmentation: ", augmentation)

Creating directory called  UNet_with_augmentation
--------------------
Configuration Setting: 
Model:  UNet
Max Epochs:  100
Batch Size:  12
Number of Workers:  8
Optimizer:  Adam
Learning Rate:  1e-05
Augmentation:  True


## Create Model

In [10]:
criterion = torch.nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else torch.nn.CrossEntropyLoss()
cudnn.benchmark = True

#Creating the model
print("Creating model...")
if name == 'NestedUNet':
    model = NestedUNet(num_classes=4)
else:
    model = UNet(n_channels=1, n_classes=4)
model = model.cuda() if torch.cuda.is_available() else model

if torch.cuda.device_count() > 1:
    print("We can use ", torch.cuda.device_count(), " GPUs.")
    model = nn.DataParallel(model)

params = filter(lambda p: p.requires_grad, model.parameters())

if optimizer == 'Adam':
    optimizer = optim.Adam(params, lr=lr, weight_decay=weight_decay)
elif optimizer == 'SGD':
    optimizer = optim.SGD(params, lr=lr, momentum=momentum, nesterov=nesterov, weight_decay=weight_decay)
else:
    raise NotImplementedError
    
summary.summary(model,(1,512,512))

Creating model...
Layer (type:depth-idx)                   Output Shape              Param #
├─inconv: 1-1                            [-1, 64, 512, 512]        --
|    └─double_conv: 2-1                  [-1, 64, 512, 512]        --
|    |    └─Sequential: 3-1              [-1, 64, 512, 512]        37,824
├─down: 1-2                              [-1, 128, 256, 256]       --
|    └─Sequential: 2-2                   [-1, 128, 256, 256]       --
|    |    └─MaxPool2d: 3-2               [-1, 64, 256, 256]        --
|    |    └─double_conv: 3-3             [-1, 128, 256, 256]       221,952
├─down: 1-3                              [-1, 256, 128, 128]       --
|    └─Sequential: 2-3                   [-1, 256, 128, 128]       --
|    |    └─MaxPool2d: 3-4               [-1, 128, 128, 128]       --
|    |    └─double_conv: 3-5             [-1, 256, 128, 128]       886,272
├─down: 1-4                              [-1, 512, 64, 64]         --
|    └─Sequential: 2-4                   [-1, 512, 64

/home/joseph.portugal/.local/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Layer (type:depth-idx)                   Output Shape              Param #
├─inconv: 1-1                            [-1, 64, 512, 512]        --
|    └─double_conv: 2-1                  [-1, 64, 512, 512]        --
|    |    └─Sequential: 3-1              [-1, 64, 512, 512]        37,824
├─down: 1-2                              [-1, 128, 256, 256]       --
|    └─Sequential: 2-2                   [-1, 128, 256, 256]       --
|    |    └─MaxPool2d: 3-2               [-1, 64, 256, 256]        --
|    |    └─double_conv: 3-3             [-1, 128, 256, 256]       221,952
├─down: 1-3                              [-1, 256, 128, 128]       --
|    └─Sequential: 2-3                   [-1, 256, 128, 128]       --
|    |    └─MaxPool2d: 3-4               [-1, 128, 128, 128]       --
|    |    └─double_conv: 3-5             [-1, 256, 128, 128]       886,272
├─down: 1-4                              [-1, 512, 64, 64]         --
|    └─Sequential: 2-4                   [-1, 512, 64, 64]         --
|

## Load Dataset

In [11]:
#directory of Images and Masks folders (generated from preprocessing)                                         
IMAGE_DIR = '/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Image/'
MASK_DIR = '/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Mask/'                                                                 

#meta information
meta = pd.read_csv('/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Meta/meta.csv')
meta = meta[meta['patient_diagnosis'] != 0]

#Get train/test label from metadata file
meta['original_image'] = meta['original_image'].apply(lambda x: IMAGE_DIR + "LIDC-IDRI-" + x[:4] + "/" + x + ".npy")
meta['mask_image'] = meta['mask_image'].apply(lambda x: MASK_DIR + "LIDC-IDRI-" + x[:4] + "/" + x + ".npy")


#Split into training and validation
train_meta = meta[meta['data_split']=='Train']
val_meta = meta[meta['data_split']=='Validation']

#Get training images into list
train_image_paths = list(train_meta['original_image'])
train_mask_paths = list(train_meta['mask_image'])

#Get validation images into list
val_image_paths = list(val_meta['original_image'])
val_mask_paths = list(val_meta['mask_image'])

print("*"*50)
print("Original images: {}, masks: {} for training.".format(len(train_image_paths),len(train_mask_paths)))
print("Original images: {}, masks: {} for validation.".format(len(val_image_paths),len(val_mask_paths)))
print("Ratio between Validation and Training is {:2f}".format(len(val_image_paths)/len(train_image_paths)))
print("*"*50)

#Creating custom LIDC dataset
train_dataset = LidcDataset(train_image_paths, train_mask_paths, transforms=toTensor)
val_dataset = LidcDataset(val_image_paths, val_mask_paths, transforms=toTensor)

train_dataset_A1 = LidcDataset(train_image_paths, train_mask_paths, transforms=transform1)
train_dataset_A2 = LidcDataset(train_image_paths, train_mask_paths, transforms=transform2)
train_dataset_A3 = LidcDataset(train_image_paths, train_mask_paths, transforms=transform3)

train_sets = torch.utils.data.ConcatDataset([train_dataset, train_dataset_A1, train_dataset_A2, train_dataset_A3])

print("*"*50)
print("Train: {}".format(len(train_sets)))
print("Validate: {}".format(len(val_dataset)))
print("*"*50)


#Creating Dataloader
train_loader = DataLoader(
  train_sets,
  batch_size=batch_size,
  shuffle=True,
  pin_memory=True,
  drop_last=True,
  num_workers=num_workers
)
val_loader = DataLoader(
  val_dataset,
  batch_size=batch_size,
  shuffle=False,
  pin_memory=True,
  drop_last=False,
  num_workers=num_workers
)

**************************************************
Original images: 980, masks: 980 for training.
Original images: 177, masks: 177 for validation.
Ratio between Validation and Training is 0.180612
**************************************************
**************************************************
Train: 3920
Validate: 177
**************************************************


## Train the Model

In [12]:
torch.cuda.empty_cache()

In [ ]:
log = pd.DataFrame(index=[], columns=['epoch','lr','loss','iou','dice','sensitivity','val_loss','val_iou'])

best_dice = 0
trigger = 0

for epoch in range(epochs):

    #Model Training
    avg_meters = {'loss': AverageMeter(), 'iou': AverageMeter(), 'dice': AverageMeter(), 'sensitivity': AverageMeter()}
    model.train()
    pbar = tqdm(total=len(train_loader)) #progress bar

    for i, data in enumerate(train_loader):

        input = data[0].cuda()
        target = data[1].cuda()
        output = model(input)

        #Get loss and metric
        loss = criterion(output, torch.argmax(target, dim=1))
        iou = iou_score_multiclass(output, target, n_classes=4)
        dice = dice_coef_multiclass(output, target, n_classes=4)
        sensitivity = sensitivity_metric_multiclass(output, target)

        #Calculate the gradient and perform optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Update average metrics
        avg_meters['loss'].update(loss.item(), input.size(0))
        avg_meters['iou'].update(iou, input.size(0))
        avg_meters['dice'].update(dice, input.size(0))
        avg_meters['sensitivity'].update(sensitivity, input.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
            ('dice', avg_meters['dice'].avg),
            ('sensitivity', avg_meters['sensitivity'].avg)
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    train_log = OrderedDict([
        ('loss', avg_meters['loss'].avg),
        ('iou', avg_meters['iou'].avg),
        ('dice', avg_meters['dice'].avg),
        ('sensitivity', avg_meters['sensitivity'].avg)
    ])


    #Model Validation
    val_avg_meters = {'val_loss': AverageMeter(), 'val_iou': AverageMeter(), 'val_dice': AverageMeter(), 'val_sensitivity': AverageMeter()}
    model.eval()

    with torch.no_grad():
        val_pbar = tqdm(total=len(val_loader))
        for i, val_data in enumerate(val_loader):

            val_input = val_data[0].cuda()
            val_target = val_data[1].cuda()
            val_output = model(val_input)

            val_loss = criterion(val_output, torch.argmax(val_target, dim=1))
            val_iou = iou_score_multiclass(val_output, val_target, n_classes=4)
            val_dice = dice_coef_multiclass(val_output, val_target, n_classes=4)
            val_sensitivity = sensitivity_metric_multiclass(val_output, val_target)

            val_avg_meters['val_loss'].update(val_loss.item(), val_input.size(0))
            val_avg_meters['val_iou'].update(val_iou, val_input.size(0))
            val_avg_meters['val_dice'].update(val_dice, val_input.size(0))
            val_avg_meters['val_sensitivity'].update(val_sensitivity, val_input.size(0))

            val_postfix = OrderedDict([
                ('val_loss', val_avg_meters['val_loss'].avg),
                ('val_iou', val_avg_meters['val_iou'].avg),
                ('val_dice', val_avg_meters['val_dice'].avg),
                ('val_sensitivity', val_avg_meters['val_sensitivity'].avg)
            ])
            val_pbar.set_postfix(val_postfix)
            val_pbar.update(1)
        val_pbar.close()

    val_log = OrderedDict([
        ('val_loss', val_avg_meters['val_loss'].avg),
        ('val_iou', val_avg_meters['val_iou'].avg),
        ('val_dice', val_avg_meters['val_dice'].avg),
        ('val_sensitivity', val_avg_meters['val_sensitivity'].avg)
    ])
    

    print('Training Epoch {}/{},  Training BCE Loss: {:.4f},  Training DICE: {:.4f},  Training IOU: {:.4f},  Training Sensitivity: {:.4f},  Validation BCE Loss: {:.4f},  Validation DICE: {:.4f},  Validation IOU: {:.4f},  Validation Sensitivity: {:.4f}'.format(
        epoch+1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], train_log['sensitivity'], val_log['val_loss'], val_log['val_dice'], val_log['val_iou'], val_log['val_sensitivity']
    ))

    #Save values to csv file
    tmp = pd.Series([
        epoch,
        lr,
        train_log['loss'],
        train_log['iou'],
        train_log['dice'],
        val_log['val_loss'],
        val_log['val_iou'],
        val_log['val_dice'],
        val_log['val_sensitivity']
    ], index=['epoch', 'lr', 'loss', 'iou', 'dice', 'val_loss', 'val_iou', 'val_dice', 'val_sensitivity'])

    log = log.append(tmp, ignore_index=True)
    log.to_csv('model_outputs/{}/log_A2.csv'.format(file_name), index=False)

    trigger += 1

    #If best DICE score, save the model
    if val_log['val_dice'] > best_dice:
        torch.save(model.state_dict(), 'model_outputs/{}/model_A2.pth'.format(file_name))
        best_dice = val_log['val_dice']
        print("Saved new best model based on DICE metric!")
        trigger = 0
    
    if early_stopping >= 0 and trigger >= early_stopping:
        print("Early stopping.")
        break

    torch.cuda.empty_cache()

100%|██████████| 15/15 [00:10<00:00,  1.46it/s, val_loss=1.22, val_iou=0.982, val_dice=5.41e-10, val_sensitivity=1]


Training Epoch 1/100,  Training BCE Loss: 1.2620,  Training DICE: 0.0000,  Training IOU: 0.9203,  Training Sensitivity: 1.0000,  Validation BCE Loss: 1.2239,  Validation DICE: 0.0000,  Validation IOU: 0.9822,  Validation Sensitivity: 1.0000
Saved new best model based on DICE metric!


100%|██████████| 15/15 [00:06<00:00,  2.34it/s, val_loss=1.2, val_iou=0.991, val_dice=2.1e-8, val_sensitivity=1]  


Training Epoch 2/100,  Training BCE Loss: 1.2004,  Training DICE: 0.0000,  Training IOU: 0.9962,  Training Sensitivity: 1.0000,  Validation BCE Loss: 1.1970,  Validation DICE: 0.0000,  Validation IOU: 0.9907,  Validation Sensitivity: 1.0000
Saved new best model based on DICE metric!


100%|██████████| 15/15 [00:06<00:00,  2.36it/s, val_loss=1.18, val_iou=0.991, val_dice=0.458, val_sensitivity=1]


...

100%|██████████| 15/15 [00:06<00:00,  2.35it/s, val_loss=0.746, val_iou=1, val_dice=1, val_sensitivity=1]


Training Epoch 67/100,  Training BCE Loss: 0.7463,  Training DICE: 1.0000,  Training IOU: 1.0000,  Training Sensitivity: 1.0000,  Validation BCE Loss: 0.7460,  Validation DICE: 1.0000,  Validation IOU: 1.0000,  Validation Sensitivity: 1.0000


100%|██████████| 15/15 [00:06<00:00,  2.32it/s, val_loss=0.746, val_iou=1, val_dice=1, val_sensitivity=1]


Training Epoch 68/100,  Training BCE Loss: 0.7460,  Training DICE: 1.0000,  Training IOU: 1.0000,  Training Sensitivity: 1.0000,  Validation BCE Loss: 0.7460,  Validation DICE: 1.0000,  Validation IOU: 1.0000,  Validation Sensitivity: 1.0000


 76%|███████▋  | 249/326 [05:11<01:35,  1.25s/it, loss=0.746, iou=1, dice=1, sensitivity=1]